## **Module**

In [1]:
from wmbio import * 

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## **Best Sub-Group Selection**

* **FILE PATH**

In [2]:
os.chdir("..")
os.getcwd()

'/home/wmbio/WORK/gitworking/Multi-omics-intergration/src'

In [3]:
# PATH
CANCER_TYPE = "LIHC"
GROUP_PHTH = os.getcwd() + '/group/'
PNG_PATH = os.getcwd() + '/png/'
GROUP_VALIDATION_PATH = os.getcwd() + '/group_validation/'
DEG_PATH = os.getcwd() + "/best_deg/"
RDATA_PATH = os.getcwd() + "/RAW_DATA/GDC_PREPARE/"
METHOD = 'deseq2'
LOGFC=1
FDR=0.05

In [8]:
# Load Validation score
col=['FILENAME','Log Rank Test','Silhouette','RNA_ANOVA_F1','RNA_RF_F1',
     'miRNA_ANOVA_F1','miRNA_RF_F1','Methylation_ANOVA_F1','Methylation_RF_F1']

group_score = pd.read_csv(GROUP_VALIDATION_PATH + 'LIHC_validation.csv', usecols=col)

# Condition for Filtering
filter_cond = (group_score['Silhouette'] >= 0.2) & (group_score['Log Rank Test'] < 0.05) & \
              ((group_score['RNA_ANOVA_F1'] > 80) | (group_score['RNA_RF_F1'] > 80)) & \
              ((group_score['miRNA_ANOVA_F1'] > 80) | (group_score['miRNA_RF_F1'] > 80)) & \
              ((group_score['Methylation_ANOVA_F1'] > 85) | (group_score['Methylation_RF_F1'] > 85))
group_score = group_score[filter_cond].sort_values(["Silhouette"], ascending = (False))

bestSubgroup = group_score.FILENAME.to_list()

# 5 fold로 제한.
if len(bestSubgroup) > 10:
    bestSubgroup = bestSubgroup[:10]

# DEA
dea_result = list()
for best_group in bestSubgroup:
    SAMPLE_GROUP = GROUP_PHTH + CANCER_TYPE + "/" + CANCER_TYPE + "_GROUP_" + best_group + ".txt"
    
    # DEG Extraction
    deg_list = deg_extract(log_fc=LOGFC, fdr=FDR,
                   cancer_type=CANCER_TYPE, 
                   sample_group=SAMPLE_GROUP, deg_path=DEG_PATH, 
                   file_name=best_group,
                   rdata_path=RDATA_PATH,
                   method=METHOD,
                   batch_removal=True)
    
    dea_result.append(deg_list)

# Filter DEA
# combine result
if METHOD == 'all':
    dea_combine = list(map(deseq2_edger_combine, dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'outer'), dea_combine)
elif METHOD == 'deseq2' :
    dea_combine = list(map(lambda d : d[["row", "log2FoldChange"]], dea_result))
    dea_combine = [col_rename(dea_combine[index], index, bestSubgroup) for index in range(len(dea_combine))]
    dea_combine = reduce(lambda left, right : pd.merge(left, right, left_on='gene', right_on='gene', how = 'inner'), dea_combine)

# median & mean
dea_combine["SubGroup-log2FC_median"] = dea_combine.iloc[:, 1:].median(axis=1)
dea_combine["SubGroup-log2FC_mean"] = dea_combine.iloc[:, 1:].mean(axis=1)

Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  

* **NT vs TP DEA(Deseq2)**

In [11]:
nt_tp_deseq2 = deg_extract_normal(log_fc=0, pvalue=0.25, cancer_type=CANCER_TYPE, 
                                  rdata_path=RDATA_PATH, deg_path=DEG_PATH, batch_removal=True)

nt_tp_deseq2_col = nt_tp_deseq2[['row', 'log2FoldChange', 'pvalue']]
nt_tp_deseq2_col.columns = ['gene', 'NT-TP_log2FoldChange', 'pvalue']

Number of significant surrogate variables is:  2 
Iteration (out of 5 ):1  2  3  4  5  

* **Write Result**

In [12]:
pd.merge(left=dea_combine, right=nt_tp_deseq2_col, left_on='gene', right_on='gene', how = 'left').to_csv('temp.csv', index = False)

In [1]:
def query_tm_db(query):
    # Connect to MariaDB (mariadb.connect 대신 pymysql.connect 로 해도 된다)
    dbconn = pymysql.connect(
        user="root",
        password="sempre813!",
        host="192.168.0.91",
        port=3306,
        database="Textmining"
    )
 
    # mariaDB Query to Pandas DataFrame
    query_result= pd.read_sql(query,dbconn)
    dbconn.close()
    
    return query_result

In [7]:
temp = pd.read_csv("../../RESULT/LIHC/LIHC-20220315-134846.csv")

In [8]:
temp

,gene,SubGroup-20220203-152500_log2FoldChange,SubGroup-20220125-002321_log2FoldChange,SubGroup-20220214-185452_log2FoldChange,SubGroup-20220216-200900_log2FoldChange,SubGroup-20220217-005003_log2FoldChange,SubGroup-20220124-145618_log2FoldChange,SubGroup-20220202-091554_log2FoldChange,SubGroup-20220128-143511_log2FoldChange,SubGroup-20220218-103449_log2FoldChange,SubGroup-20220127-164231_log2FoldChange,SubGroup-log2FC_median,SubGroup-log2FC_mean,NT-TP_log2FoldChange,pvalue
0,NR0B1,2.264361,2.080394,2.228180,3.257259,3.700563,2.550543,2.119919,1.966184,2.239832,2.055414,2.234006,2.426969,NaN,NaN
1,ALDH1A3,-1.793150,-1.682205,-1.902537,-1.998898,-1.275650,-1.752688,-1.598957,-1.743267,-1.587014,-1.624524,-1.712736,-1.697421,NaN,NaN
2,BPI,2.348355,2.310937,2.184856,1.767478,1.553725,2.263871,2.262357,2.379578,2.027197,2.140075,2.223607,2.132912,1.583778,0.000303
3,BRDT,5.201566,4.593574,4.692542,8.256799,8.563013,4.222280,5.068229,4.741670,5.348424,4.853681,4.960955,5.500249,NaN,NaN
4,BUB1B,1.337425,1.304906,1.309792,1.405513,1.321236,1.262807,1.348068,1.074070,1.156726,1.047042,1.307349,1.261358,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
265,GAGE12J,3.785426,2.435323,2.656644,3.371597,4.477213,2.424847,3.797834,3.968996,4.343834,2.814784,3.578511,3.423183,2.625280,0.003173
266,GAGE2D,4.052665,3.730318,4.087134,5.683297,4.969525,4.004418,4.025910,4.894874,4.226536,3.829911,4.069900,4.324953,3.418461,0.001889
267,CECR7,2.775261,2.523702,2.685346,2.517688,2.484935,2.366181,2.602169,2.360199,2.395819,2.386686,2.501311,2.509027,NaN,NaN
268,LOC100131726,-1.217191,-1.413597,-1.433826,-2.740009,-2.370617,-2.447118,-1.350025,-1.544879,-1.617089,-1.649765,-1.580984,-1.760464,1.194093,0.015637


In [36]:
sql = 'select * from LIHC'
df = query_tm_db(sql)

/home/wmbio/anaconda3/envs/multiomics-cpu/lib/python3.8/site-packages/pandas/io/sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [38]:
temp2 = pd.merge(left=temp, right=df, left_on="gene", right_on="gene", how='left')

In [42]:
temp2.to_csv("temp.csv")